<a href="https://colab.research.google.com/github/heltonleiras/Downloader-torrent-para-o-Google-Drive-heltonleiras/blob/main/Downloader_torrent_para_o_Google_Drive_%7C%7C%40heltonleiras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Downloader de torrent para o Google Drive | Siga @heltonleiras

**Nota importante:**
> Você terá cerca de 384 GB para baixar qualquer torrent que desejar.

### Instale o libtorrent e inicialize a sessão | Siga @heltonleiras

In [ ]:
!python -m pip install --upgrade pip setuptools wheel
!python -m pip install lbry-libtorrent
!apt install python3-libtorrent

import libtorrent as lt

ses = lt.session()
ses.listen_on(6881, 6891)
downloads = []

### Monte o Google Drive | Siga @heltonleiras
Para transmitir arquivos, precisamos montar o Google Drive.

In [ ]:
from google.colab import files

source = files.upload()
params = {
    "save_path": "/content/drive/My Drive/Torrent",
    "ti": lt.torrent_info(list(source.keys())[0]),
}
downloads.append(ses.add_torrent(params))

### Adicionar do link magnético | Siga @heltonleiras
Você pode executar esta célula para adicionar mais arquivos quantas vezes quiser

In [ ]:
rom google.colab import drive

drive.mount("/content/drive")

### Fazer upload de magnet link se quiser | Siga @heltonleiras
Você pode executar esta célula para adicionar mais arquivos quantas vezes quiser

In [ ]:
params = {"save_path": "/content/drive/My Drive/Torrent"}

while True:
    magnet_link = input("Insira o link magnético ou digite Sair: ")
    if magnet_link.lower() == "sair":
        break
    downloads.append(
        lt.add_magnet_uri(ses, magnet_link, params)
    )


### Iniciar download | Siga @heltonleiras

In [ ]:
import time
from IPython.display import display
import ipywidgets as widgets

state_str = [
    "na fila",
    "verificando",
    "baixando metadados",
    "baixando",
    "finalizado",
    "semeando",
    "alocando",
    "verificando currículo rápido",
]

layout = widgets.Layout(width="auto")
style = {"description_width": "initial"}
download_bars = [
    widgets.FloatSlider(
        step=0.01, disabled=True, layout=layout, style=style
    )
    for _ in downloads
]
display(*download_bars)

while downloads:
    next_shift = 0
    for index, download in enumerate(downloads[:]):
        bar = download_bars[index + next_shift]
        if not download.is_seed():
            s = download.status()

            bar.description = " ".join(
                [
                    download.name(),
                    str(s.download_rate / 1000),
                    "kB/s",
                    state_str[s.state],
                ]
            )
            bar.value = s.progress * 100
        else:
            next_shift -= 1
            ses.remove_torrent(download)
            downloads.remove(download)
            bar.close() # Siga meu instagram @heltonleiras
            download_bars.remove(bar)
            print(download.name(), "complete")
    time.sleep(1)
